<a href="https://colab.research.google.com/github/adityapattar3322/Crop-Disease-Detection-App/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json

# Define constants
IMG_WIDTH, IMG_HEIGHT = 128, 128
BATCH_SIZE = 32
TRAIN_DIR = 'data/train'
VALIDATION_DIR = 'data/validation'

# 1. Image Data Augmentation and Loading
print("Preparing data generators...")
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# 2. Build the CNN Model
print("Building the CNN model...")
model = Sequential([
    # First convolutional block
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    MaxPooling2D(2, 2),

    # Second convolutional block
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    # Third convolutional block
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    # Flatten the results to feed into a DNN
    Flatten(),

    # Dense layer with dropout for regularization
    Dense(512, activation='relu'),
    Dropout(0.5),

    # Output layer
    Dense(train_generator.num_classes, activation='softmax')
])

# 3. Compile the Model
print("Compiling the model...")
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

# 4. Train the Model
print("Training the model...")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=25, # Adjust epochs based on performance
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

# 5. Save the Model and Class Indices
print("Saving the model and class labels...")

# Save the entire model to a HDF5 file.
model.save('crop_disease_model.h5')

# Save the class labels (disease names) to a JSON file
# The keys are the integer indices, and values are the class names
class_indices = train_generator.class_indices
# Invert the dictionary to map from index to class name
labels = {v: k for k, v in class_indices.items()}

with open('class_labels.json', 'w') as f:
    json.dump(labels, f)

print("Training complete and artifacts saved!")

Preparing data generators...


FileNotFoundError: [Errno 2] No such file or directory: 'data/train'